The current script is used to extract event codes from .csv files produced by PsychoPy and format them in .txt files that FSL can utilize for first level analyses.

Note: This script is currently written with the assumption that all data is located in the same folder

Import libraries

In [1]:
import openpyxl as xl
import xlrd
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import csv
import os

Establish variables and make directories (These are the only aspects of the script that may need to change)

In [3]:
folder_name = 'combined_orders'               
auditory_folder = 'auditory_folder'
auditory_times = pd.read_excel("EV_timing.xlsx", 'Auditory_Times', index = False)
speech_struct = 'speech_struct'
tone_rand = 'tone_rand'             
tone_struct = 'tone_struct'
speech_rand = 'speech_rand'

os.mkdir(speech_struct)
os.mkdir(tone_rand)
os.mkdir(tone_struct)
os.mkdir(speech_rand)

os.makedirs(auditory_folder) #Creat a new folder to save new auditory excel files
auditory_files = []


Run program to extract raw event files, create a list that stores event information in individual files, adds timing information and sorts into files compatible with FSL.

In [4]:
for i in os.listdir(folder_name): #load every file from the folder
    if i.endswith('auditory_1.csv') or i.endswith('auditory_3.csv'):
        auditory_files.append(i)


def save_new_folder(files, new_folder_name):
    for list in files:

        file = open(os.path.join(folder_name, list))  # load csv into the python
        numline = len(file.readlines())

        def read_cell(x, y):  # function that can read the cell
            with open(os.path.join(folder_name, list), 'r') as f:
                reader = csv.reader(f)
                y_count = 0
                for n in reader:
                    if y_count == y:
                        cell = n[x]
                        return cell
                    y_count += 1

        a = []  # creat a list to store R S B
        for i in range(1, numline + 1):
            a.append(read_cell(5, i))
        # cell = sheet.cell(row, 2)
        # for list in cell.value:
        #     a.append(list)

        inRow = []  # create another list to store R and S without repeat
        last = None
        for x in a:
            if last != x:
                inRow.append(x)
            last = x

        inRow = [x for x in inRow if x != 'B']  # Delete all the B's
        inRow = [x for x in inRow if x != None]
        print(inRow)

        # for f in os.listdir(folder_name): #rename the new excel file which is .xlsx file
        f_name, f_ext = os.path.splitext(list)
        new_ext = '.xlsx'
        new_name = '{}{}'.format(f_name, new_ext)

        df = pd.DataFrame(inRow)  # write the result into a new excel
        writer = ExcelWriter(os.path.join(new_folder_name, new_name))  # save every file to new folder
        df.to_excel(writer, 'Sheet1', index=False)
        writer.save()

save_new_folder(auditory_files, auditory_folder)

def save_time(folder, time):
    for i in os.listdir(folder):
        book = load_workbook(os.path.join(folder, i))
        writer = pd.ExcelWriter(os.path.join(folder, i), engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        time.to_excel(writer, 'Sheet1', index=False, startrow=1, startcol=1)
        writer.save()

save_time(auditory_folder,auditory_times)

['R', 'S', 'S', 'R', 'R', 'S']
['S', 'R', 'R', 'S', 'R', 'S']
['R', 'S', 'R', 'S', 'S', 'R']
['R', 'R', 'S', 'S', 'S', 'R']
['R', 'S', 'R', 'R', 'S', 'S']
['S', 'R', 'R', 'S', 'R', 'S']
['S', 'R', 'S', 'R', 'R', 'S']
['S', 'S', 'S', 'R', 'R', 'R']
['R', 'S', 'R', 'S', 'R', 'S']
['S', 'R', 'S', 'R', 'S', 'R']
['S', 'R', 'R', 'S', 'S', 'R']
['R', 'S', 'R', 'S', 'R']
['R', 'S', 'R', 'S', 'S', 'R']
['S', 'R', 'S', 'S', 'R', 'R']
['S', 'R', 'S', 'R', 'S', 'R']
['S', 'S', 'S', 'R', 'R', 'R']
['S', 'R', 'S', 'S', 'R', 'R']
['R', 'S', 'R', 'S', 'R', 'S']
['R', 'R', 'S', 'R', 'S', 'S']
['S', 'S', 'R', 'S', 'R', 'R']
['S', 'R', 'S', 'S', 'R', 'R']
['S', 'R', 'S', 'S', 'R', 'R']
['S', 'R', 'R', 'R', 'S', 'S']
['S', 'S', 'R', 'R', 'R', 'S']


If any output did not have generate 6 EV codes (e.g. there should be 3 'R' and 3 'S' in each row of the output), then those files need to be removed from the folder (and likely computed manually). The below code will only work on complete files.

In [5]:
auditory_file_1 = []
auditory_file_3 = []

for i in os.listdir(auditory_folder): #load every file from the folder
    if i.endswith('auditory_1.xlsx'):
        auditory_file_1.append(i)
    elif i.endswith('auditory_3.xlsx'):
        auditory_file_3.append(i)


def keep_S(files, folder,folder_name):
    for i in files:
        df = pd.read_excel(os.path.join(folder_name,i),index_col=0)
        df = df.drop(["R"])
        # df = df[:]
        #df = df.drop(df.columns[0],axis=1)
        print(df)
        f_name, f_ext = os.path.splitext(i)
        new_ext = '.txt'
        new_name = '{}{}'.format(f_name, new_ext)
        with open(os.path.join(folder, new_name),'w') as outfile:
            df.to_string(outfile, index=False, header=None)

def keep_R(files, folder,folder_name):
    for i in files:
        df = pd.read_excel(os.path.join(folder_name,i),index_col=0)
        df = df.drop(["S"])
        # df = df[:]
        #df = df.drop(df.columns[0],axis=1)
        print(df)
        f_name, f_ext = os.path.splitext(i)
        new_ext = '.txt'
        new_name = '{}{}'.format(f_name, new_ext)
        with open(os.path.join(folder, new_name),'w') as outfile:
            df.to_string(outfile, index=False, header=None)

keep_S(auditory_file_1, speech_struct,auditory_folder)
keep_R(auditory_file_1, tone_rand,auditory_folder)
keep_S(auditory_file_3, tone_struct,auditory_folder)
keep_R(auditory_file_3, speech_rand,auditory_folder)


   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
S    116.6696     46.0524           1
S    229.3153     46.0515           1
S    285.6353     46.0541           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
S    116.6696     46.0524           1
S    172.9922     46.0524           1
S    229.3153     46.0515           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
S      4.0905     45.9648           1
S    172.9922     46.0524           1
S    229.3153     46.0515           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
S      4.0905     45.9648           1
S     60.8216     45.5779           1
S    116.6696     46.0524           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
S    116.6696     46.0524           1
S    229.3153     46.0515           1
S    285.6353     46.0541           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0           

   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
R     60.8216     45.5779           1
R    172.9922     46.0524           1
R    285.6353     46.0541           1
   Unnamed: 1  Unnamed: 2  Unnamed: 3
0                                    
R    116.6696     46.0524           1
R    229.3153     46.0515           1
R    285.6353     46.0541           1
